In [20]:
import os
import csv
import sys
import re

In [2]:
from surprise import Dataset
from surprise import Reader

In [3]:
from collections import defaultdict
import numpy as np

In [14]:
class MovieLens:

    movieID_to_name = {}
    name_to_movieID = {}
    
    ratingsPath = 'C:/Users/Laci/anaconda_envs/RecSys/ml-latest-small/ratings.csv'
    moviesPath = 'C:/Users/Laci/anaconda_envs/RecSys/ml-latest-small/movies.csv'
    
    def loadMovieLensLatestSmall(self):
        ratingsDataset = 0
        self.movieID_to_name = {}
        self.name_to_movieID = {}
        
        # userId movieId rating timestamp
        reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)

        ratingsDataset = Dataset.load_from_file(self.ratingsPath, reader=reader)

        with open(self.moviesPath, newline='', encoding='ISO-8859-1') as csvfile:
                movieReader = csv.reader(csvfile)
                next(movieReader)  #Skip header line
                
                for row in movieReader:
                    movieID = int(row[0])
                    movieName = row[1]
                    self.movieID_to_name[movieID] = movieName
                    self.name_to_movieID[movieName] = movieID

        return ratingsDataset
    

    def getUserRatings(self, user):
        userRatings = []
        hitUser = False
        
        with open(self.ratingsPath, newline='') as csvfile:
            ratingReader = csv.reader(csvfile)
            next(ratingReader)
            
            for row in ratingReader:
                userID = int(row[0])
                if (user == userID):
                    movieID = int(row[1])
                    rating = float(row[2])
                    userRatings.append((movieID, rating))
                    hitUser = True
                if (hitUser and (user != userID)):
                    break

        return userRatings

    
    def getPopularityRanks(self):
        ratings = defaultdict(int)
        rankings = defaultdict(int)
        
        with open(self.ratingsPath, newline='') as csvfile:
            ratingReader = csv.reader(csvfile)
            next(ratingReader)
            
            for row in ratingReader:
                movieID = int(row[1])
                ratings[movieID] += 1
                
        rank = 1
        for movieID, ratingCount in sorted(ratings.items(), key=lambda x: x[1], reverse=True):
            rankings[movieID] = rank
            rank += 1
            
        return rankings
  

    def getGenres(self):
        genres = defaultdict(list)
        genreIDs = {}
        maxGenreID = 0
        
        with open(self.moviesPath, newline='', encoding='ISO-8859-1') as csvfile:
            movieReader = csv.reader(csvfile)
            next(movieReader)  #Skip header line
            
            for row in movieReader:
                movieID = int(row[0])
                genreList = row[2].split('|')
                genreIDList = []
                for genre in genreList:
                    if genre in genreIDs:
                        genreID = genreIDs[genre]
                    else:
                        genreID = maxGenreID
                        genreIDs[genre] = genreID
                        maxGenreID += 1
                        
                    genreIDList.append(genreID)
                    
                genres[movieID] = genreIDList
                
        # Convert integer-encoded genre lists to bitfields that we can treat as vectors
        for (movieID, genreIDList) in genres.items():
            bitfield = [0] * maxGenreID
            
            for genreID in genreIDList:
                bitfield[genreID] = 1
                
            genres[movieID] = bitfield            
        
        return genres
    
    
    def getYears(self):
        p = re.compile(r"(?:\((\d{4})\))?\s*$")
        years = defaultdict(int)
        
        with open(self.moviesPath, newline='', encoding='ISO-8859-1') as csvfile:
            movieReader = csv.reader(csvfile)
            next(movieReader)
            
            for row in movieReader:
                movieID = int(row[0])
                title = row[1]
                m = p.search(title)
                year = m.group(1)
                if year:
                    years[movieID] = int(year)
                    
        return years
    
    
    def getMovieName(self, movieID):
        if movieID in self.movieID_to_name:
            return self.movieID_to_name[movieID]
        else:
            return ""
        
    def getMovieID(self, movieName):
        if movieName in self.name_to_movieID:
            return self.name_to_movieID[movieName]
        else:
            return 0

In [15]:
ml = MovieLens()

In [16]:
data = ml.loadMovieLensLatestSmall()

In [ ]:
ml.movieID_to_name

In [ ]:
ml.name_to_movieID

In [ ]:
ml.getYears()

In [ ]:
ml.getGenres()

In [ ]:
ml.getPopularityRanks()

In [41]:
# Test User Id
testSubject = 85

userRatings = ml.getUserRatings(testSubject)

In [ ]:
userRatings

In [43]:
len(userRatings)

107

In [44]:
from surprise import SVD

In [45]:
def BuildAntiTestSetForUser(testSubject, trainset):    
    fill = trainset.global_mean

    anti_testset = []
    
    u = trainset.to_inner_uid(str(testSubject))
    
    user_items = set([j for (j, _) in trainset.ur[u]])
    anti_testset += [(trainset.to_raw_uid(u), trainset.to_raw_iid(i), fill) for
                             i in trainset.all_items() if
                             i not in user_items]
    return anti_testset

In [46]:
userRatings = ml.getUserRatings(testSubject)
loved = []
hated = []

for ratings in userRatings:
    if (float(ratings[1]) > 4.0):
        loved.append(ratings)
    if (float(ratings[1]) < 3.0):
        hated.append(ratings)

In [47]:
print("\nUser ", testSubject, " loved these movies:")
for ratings in loved:
    print(ml.getMovieName(ratings[0]))
    
print("\n...and didn't like these movies:")
for ratings in hated:
    print(ml.getMovieName(ratings[0]))


User  85  loved these movies:
Jumanji (1995)
GoldenEye (1995)
Braveheart (1995)
Jerky Boys, The (1995)
LÃ©on: The Professional (a.k.a. The Professional) (LÃ©on) (1994)
Pulp Fiction (1994)
Stargate (1994)
Shawshank Redemption, The (1994)
Star Trek: Generations (1994)
Clear and Present Danger (1994)
Speed (1994)
True Lies (1994)
Fugitive, The (1993)
Jurassic Park (1993)
Terminator 2: Judgment Day (1991)
Mission: Impossible (1996)
Rock, The (1996)

...and didn't like these movies:
Grumpier Old Men (1995)
Mortal Kombat (1995)
Postman, The (Postino, Il) (1994)
Casper (1995)
Lord of Illusions (1995)
Mighty Morphin Power Rangers: The Movie (1995)
Prophecy, The (1995)
Dolores Claiborne (1995)
Heavenly Creatures (1994)
Little Women (1994)
Miracle on 34th Street (1994)
Nell (1994)
Poison Ivy II (1996)
Tank Girl (1995)
While You Were Sleeping (1995)
Wes Craven's New Nightmare (Nightmare on Elm Street Part 7: Freddy's Finale, A) (1994)
Naked Gun 33 1/3: The Final Insult (1994)
Richie Rich (1994)


In [48]:
# For recommendation model
trainSet = data.build_full_trainset()

In [49]:
algo = SVD()
algo.fit(trainSet)

In [50]:
testSet = BuildAntiTestSetForUser(testSubject, trainSet)
predictions = algo.test(testSet)

In [51]:
recommendations = []

print ("\nWe recommend:")
for userID, movieID, actualRating, estimatedRating, _ in predictions:
    intMovieID = int(movieID)
    recommendations.append((intMovieID, estimatedRating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(ml.getMovieName(ratings[0]))


We recommend:
All the President's Men (1976)
American Beauty (1999)
Usual Suspects, The (1995)
Swingers (1996)
Graduate, The (1967)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
In the Name of the Father (1993)
Chinatown (1974)
Professional, The (Le professionnel) (1981)
Wallace & Gromit: A Close Shave (1995)
